In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Bike Sharing Demand 베이스라인 잡기 1.38정도 점수
# 베이스라인 잡은 후 날짜 정보 추가해서 점수 계산해오기 0.4XXX

In [ ]:
train = pd.read_csv('/kaggle/input/bike-sharing-demand/train.csv')
train

In [ ]:
test = pd.read_csv('/kaggle/input/bike-sharing-demand/test.csv')
test

In [ ]:
train_2 = train.drop(columns = ['datetime','casual','registered','count'])
train_2

In [ ]:
test_2 = test.drop(columns = 'datetime')
test_2

In [ ]:
%%time
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_jobs=-1,n_estimators=1000)
rf.fit(train_2,train['count'])
result = rf.predict(test_2)
result

In [ ]:
sub = pd.read_csv('/kaggle/input/bike-sharing-demand/sampleSubmission.csv')
sub

In [ ]:
sub['count'] = result
sub

In [ ]:
sub.to_csv('result1.csv',index=False)

# 이렇게 베이스라인 잡았더니 1.38 정도의 결과가 나왔음

In [ ]:
# 이제부터는 날짜정보 추가하기!

In [ ]:
train['datetime'] = pd.to_datetime(train['datetime'])
train['year'] = train['datetime'].dt.year
train['month'] = train['datetime'].dt.month
train['day'] = train['datetime'].dt.day
train['hour'] = train['datetime'].dt.hour

In [ ]:
test['datetime'] = test['datetime'].astype('datetime64')
test['year'] = test['datetime'].dt.year
test['month'] = test['datetime'].dt.month
test['day'] = test['datetime'].dt.day
test['hour'] = test['datetime'].dt.hour

In [ ]:
# 빨간색으로 뜨는 warning 없애기
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 밑그림 그리는 라이브러리 가져오기
# hour 컬럼이 count 컬럼에 어떤 영향을 주는지 알아보기
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(16,8))
sns.boxplot(train['hour'],train['count'])
# 결과 : 오전 7,8시 시간대에 많이 빌린다(count 400쯤 있는 선 : 중앙값(50%))
#       박스 내부에 있는 맨 위에 선(600)--> 상위 25%
#       박스 아래쪽 선 --> 하위 25%
# 맨 위에 선 아래선(850/0 쯤 있는 값) --> 최대/최소값 
# 오전 8시에는 이용량 편차가 상당한 편!
# 저녁 8시쯤 최대값 위에 있는 점? --> outlier(이상치,극단값)  : 표준편차 +-3 에서 넘어가는 애들!
# 10시부터 15시까지 outlier가 너무 많은데...? --> 체크해주어야한다!
# ---> 왜 이때 많은가? => 많이 안 빌린다고 생각하는데 주말의 count 값들이 outlier에 들어가는 것!
# ---> 주중과 주말데이터를 나눠서(조건을 줘서) 그림을 따로 그려야 한다!

In [ ]:
# 주중 데이터프레임
weekday_df = train[train['workingday'] == 1]
weekday_df

In [ ]:
# 주말 데이터프레임
weekend_df = train[train['workingday'] == 0]
weekend_df

In [ ]:
plt.figure(figsize = (16,8))
sns.boxplot(weekday_df['hour'],weekday_df['count'])
# 이상치가 많이 줄어든 것 확인

In [ ]:
%%time
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_jobs=-1)
rf.fit(train_2,train['count'])
result = rf.predict(test_2)
result

In [ ]:
plt.figure(figsize = (16,8))
sns.boxplot(weekend_df['hour'],weekend_df['count'])
# 주말에는 10~15시에도 많이 사용하는 것을 알 수 있다!

In [ ]:
plt.figure(figsize = (16,8))
sns.boxplot(train['year'],train['count'])
# 2011년보다 2012년에 더 많이 타는 것을 알 수 있다!

In [ ]:
# day컬럼을 모델에 추가시 왜 점수가 떨어지나?
plt.figure(figsize = (16,8))
sns.boxplot(train['day'],train['count'])
# 1) train 데이터셋의 day 컬럼의 값은 19일까지 나온다
# 2) test 데이터셋은? --> 아래 셀 확인

In [ ]:
# test 데이터셋의 day컬럼 종류 확인
test['day'].unique()
# --> train과 test의 day종류가 다르므로 점수가 낮아지는 것!
# Tree 모델에서는 컬럼이 수치형인지 범주형인지 알아서 분류한다
# 따라서 day컬럼도 범주형으로 분류했다 --> 여기서 전제는 컬럼의 데이터범주가 어느정도 일치해야 하는데
# 그렇지 못해서 train에서 학습한 것을 test에 제대로 못 써먹음
# --> 점수 하락
# day 컬럼이 수치형이었다면 겹치지 않았어도 모델 점수향상에 도움을 줬을것!

In [ ]:
# 이제 점수 향상에 도움이 되는 컬럼은 hour, month, year라는 것 확인
# train['datetime'] = pd.to_datetime(train['datetime'])
train['datetime'] = train['datetime'].astype('datetime64')
train['year'] = train['datetime'].dt.year
train['month'] = train['datetime'].dt.month
train['hour'] = train['datetime'].dt.hour
train['dayofweek'] = train['datetime'].dt.dayofweek    # 0/1/2/3/4/5/6 --> 월/화/수/목/금/토/일
train

In [ ]:
test['year'] = test['datetime'].dt.year
test['month'] = test['datetime'].dt.month
test['hour'] = test['datetime'].dt.hour
test['dayofweek'] = test['datetime'].dt.dayofweek
test

In [ ]:
# dayofweek 컬럼에 대해서도 한번 시각화해봄
plt.figure(figsize=(16,8))
sns.boxplot(train['dayofweek'],train['count'])

In [ ]:
train_2 = train.drop(columns = ['day','month','datetime','casual','count','registered'])
train_2

In [ ]:
test_2 = test.drop(columns = ['day','datetime','month'])
test_2

In [ ]:
'''
# 1) rf에 np.log값을 줘서 분배 고르게
%%time
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_jobs=-1)
rf.fit(train_2,np.log(train['count']))
## 2) xgboost 
from xgboost import XGBRegressor
xgbr = XGBRegressor(learning_rate=0.1)
rf.fit(train_2,np.log(train['count']))
# learning_rate -> 학습속도 결정(디폴트 : 0.3)
# learning_rate 낮추는 이유 --> 인간이 공부할때를 생각하자!
# gradient boosting 모델 
# gradient : 미분(최적의 값 찾기) --> y = x^2 중 y값이 최소가 되는 지점을 찾아나가는 과정
# 처음엔 아무x나 잡고 학습 --> 학습해나가면서 계속 반복 --> learning_rate가 3이면 너무 빠른 반복
# --> y값을 최소로 만드는 최적 x를 지나칠 수 있음
# learning_rate를 줄이면 학습시간은 느려져도 최적의 x값을 확실히 학습가능!
'''
from catboost import CatBoostRegressor
cb = CatBoostRegressor(verbose=100)
cb.fit(train_2,np.log(train['count']))

In [ ]:
'''
result = rf.predict(test_2)
result
'''
'''
result = xgbr.predict(test_2)
result
'''
result = cb.predict(test_2)
result

In [ ]:
sub = pd.read_csv('/kaggle/input/bike-sharing-demand/sampleSubmission.csv')
sub

In [ ]:
sub['count'] = np.exp(result)
sub

In [ ]:
# holiday --> 공휴일일때 자전거를 더 빌려갈 것이다!
# --> 모델이 이렇게 학습했을것
# train / test 데이터셋의 분포가 비슷할때만 좋은 정보
# 하지만 우리 데이터셋의 day 분포는 달랐음!

# 미국 공휴일은 bike_sharing 많이 안함(count가 낮음)
# test 데이터셋에서 holiday 컬럼 특정 날짜에 직접 접근 --> count를 낮춘다!
# --> 모델이 공휴일에는 많이 쓴다고 생각했기 때문에 count 값을 낮춰준다!
# --> 0.5가 그나마 좋은 비율



#크리스마스
sub.iloc[6332:6354, 1]= sub.iloc[6332:6354, 1]*0.5
sub.iloc[3065:3087, 1]= sub.iloc[3065:3087, 1]*0.5
#추수감사절
sub.iloc[5992:6015, 1]= sub.iloc[5992:6015, 1]*0.5
sub.iloc[2771:2794, 1]= sub.iloc[2771:2794, 1]*0.5
#미국 현충일
sub.iloc[1258:1269, 1]= sub.iloc[1258:1269, 1]*0.5
sub.iloc[4492:4515, 1]= sub.iloc[4492:4515, 1]*0.5
#크리스마스 이브
sub.iloc[6308:6330, 1]= sub.iloc[6308:6330, 1]*0.5
sub.iloc[3041:3063, 1]= sub.iloc[3041:3063, 1]*0.5
sub

In [ ]:
sub.to_csv('result_2.csv',index = False)

In [ ]:
import seaborn as sns
sns.distplot(train['count'])
# 왼쪽에 많이 분포되어 있고 우측으로 꼬리가 길다!

In [ ]:
sns.distplot(np.log(train['count']))

In [ ]:
# xgboost 모델을 옵션설정만 잘 해주면 더 좋은 성능을 내려나?